In [1]:
import pandas as pd
import itertools as it
import gurobipy as gp
from gurobipy import GRB

In [4]:
data = [1.5, 1.5, 1.5, 0.75,
        1.5, 2.0, 1.5, 0.75,
        1.0, 1.0, 0.75, 0.5,
        0.75, 0.75, 0.5, 0.25,
        4.0, 4.0, 2.0,
        3.0, 3.0, 1.0,
        2.0, 2.0, 0.5,
        12.0, 6.0,
        5.0, 4.0,
        6.0]

levels = list(range(1,5))

blocks = []
for l in levels:
    horiz = list(range(1,6-l))
    blocks.extend(list(it.product([l], horiz, horiz)))

block_above = {}
for b in blocks:
    if b[0]==1: 
        block_above[b] = []
        continue
    above = [(b[0]-1, b[1],   b[2]),
             (b[0]-1, b[1]+1, b[2]),
             (b[0]-1, b[1],   b[2]+1), 
             (b[0]-1, b[1]+1, b[2]+1)]
    block_above[b] = above

whole_value  = 200000
block_value = dict(zip(blocks, [d/100*whole_value for d in data]))
extract_cost = dict(zip(levels, [3000, 6000, 8000, 10000]))

In [20]:
model = gp.Model('Opencast Mining')

# add vars
b = model.addVars(blocks,
                  ub=1,
                  name='b')

# objective function
model.setObjective((gp.quicksum(b[i]*(block_value[i]-extract_cost[i[0]]) 
                                for i in blocks)),
                   GRB.MAXIMIZE)

# add constraints
for i in blocks:
    model.addConstrs((b[a] - b[i] >= 0 for a in block_above[i]),
                     name='above'+str(i))
    
model.update()
model.write('Opencast Mining.lp')
model.optimize()

Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 56 rows, 30 columns and 112 nonzeros
Model fingerprint: 0xebc1bcea
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+03, 2e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [0e+00, 0e+00]
Presolve removed 22 rows and 12 columns
Presolve time: 0.02s
Presolved: 34 rows, 18 columns, 68 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.3014500e+04   8.003000e+00   0.000000e+00      0s
       8    1.7500000e+04   0.000000e+00   0.000000e+00      0s

Solved in 8 iterations and 0.03 seconds
Optimal objective  1.750000000e+04


In [22]:
print('Total profit of ${:.2f}'.format(model.ObjVal))
print('Extract blocks')
for i in blocks:
    if b[i].x>0:
        print(i)

Total profit of $17500.00
Extract blocks
(1, 1, 1)
(1, 1, 2)
(1, 1, 3)
(1, 2, 1)
(1, 2, 2)
(1, 2, 3)
(1, 3, 1)
(1, 3, 2)
(1, 3, 3)
(2, 1, 1)
(2, 1, 2)
(2, 2, 1)
(2, 2, 2)
(3, 1, 1)
